In [1]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
hospital_data = pd.read_excel('Hospital_dataset.xlsx', sheet_name='Hospital ER')

In [3]:
hospital_data_copy = hospital_data.copy()

In [14]:
hospital_data_copy.head(10)

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,Full_Name,Doctor,patient_admin_flag,patient_waittime,department_referral,revenue,patient_category
0,2020-03-20 08:47:00,145-39-5406,Male,69,10.0,H,Glasspool,H.Glasspool,Dr. Ben Carson,False,39,Gastroenterology,260,Senior
1,2020-06-15 11:29:00,316-34-3057,Male,4,10.0,X,Methuen,X.Methuen,Dr. Mohamed Oz,True,27,Renal,275,Child
2,2020-06-20 09:13:00,897-46-3852,Female,56,9.0,P,Schubuser,P.Schubuser,Dr. Patricia Bath,True,55,General Practice,260,Adult
3,2020-02-04 22:34:00,358-31-9711,Female,24,8.0,U,Titcombe,U.Titcombe,Dr. Jennifer Doudna,True,31,General Practice,460,Young Adult
4,2020-09-04 17:48:00,289-26-0537,Male,5,4.0,Y,Gionettitti,Y.Gionettitti,Dr. Joycelyn Elders,False,10,Orthopedics,275,Child
5,2019-04-20 00:13:00,255-51-2877,Male,58,0.0,H,Buff,H.Buff,Dr. Paul Farmer,False,59,General Practice,260,Adult
6,2019-08-23 08:26:00,465-97-0990,Female,68,10.0,F,Perrat,F.Perrat,Dr. Eric Topol,True,43,Cardiology,260,Senior
7,2019-07-29 16:57:00,157-31-7520,Female,47,8.0,K,Gwillim,K.Gwillim,Dr. Lisa Sanders,True,23,Gastroenterology,460,Adult
8,2020-02-19 06:54:00,432-34-5614,Female,79,1.0,E,Dewhirst,E.Dewhirst,Dr. Francis Collins,False,42,Physiotherapy,260,Senior
9,2020-10-11 05:25:00,609-17-8678,Male,62,4.0,M,Crebo,M.Crebo,Dr. Joycelyn Elders,False,51,Physiotherapy,260,Senior


In [5]:
hospital_data_copy['patient_gender'] = hospital_data_copy['patient_gender'].astype('category')
hospital_data_copy['patient_first_inital'] = hospital_data_copy['patient_first_inital'].astype('category')
hospital_data_copy['Doctor'] = hospital_data_copy['Doctor'].astype('category')
hospital_data_copy['department_referral'] = hospital_data_copy['department_referral'].astype('category')

In [6]:
hospital_data_copy['Full_Name'] = hospital_data_copy['Full_Name'].str.slice(0,1) +"."+ hospital_data_copy['Full_Name'].str.slice(1,)

In [7]:
hospital_data_copy['patient_sat_score'].fillna(np.NaN, inplace=True)

In [8]:
from imputing_method import sat_imputer

In [9]:
grouped_by_doc = hospital_data_copy.groupby('Doctor')

In [10]:
hospital_data_copy['patient_sat_score'] = hospital_data_copy['patient_sat_score'] = hospital_data_copy.apply(
    lambda row: sat_imputer(row, grouped_by_doc) if pd.isna(row['patient_sat_score']) else row['patient_sat_score'],
    axis=1
)

In [11]:
labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']
bins = [0, 13, 20, 36, 60, np.inf]
hospital_data_copy['patient_category'] = pd.cut(hospital_data_copy['patient_age'], bins=bins,labels=labels)

In [12]:
from imputing_method import imputer

In [13]:
hospital_data_copy['department_referral'] = hospital_data_copy['department_referral'] = hospital_data_copy.apply(
    lambda row: imputer(row, grouped_by_doc) if pd.isna(row['department_referral']) else row['department_referral'],
    axis=1
)